<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!nvidia-smi

Fri Jun 10 22:16:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    44W / 250W |      0MiB / 16280MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Inicializacion

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
import os 
os.chdir('/content/')

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git checkout .
!git pull

fatal: destination path 'TrackNet-Padel' already exists and is not an empty directory.
Already up to date.


In [4]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 4.2 MB/s 
     |████████████████████████████████| 6.7 MB 22.4 MB/s 
     |████████████████████████████████| 3.3 MB 26.9 MB/s 
     |████████████████████████████████| 9.8 MB 63.1 MB/s 
     |████████████████████████████████| 34.5 MB 1.2 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=02eed21140753bb1495f4e512828a78e7bedeeb26eb21f838203c6e313c50a5e
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled sta

### Predict

In [ ]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean4'
for i in range(7):
  video = 'raw_data/points/cleanVal/PremierRoma_Semis_LGBC_3_42_'+str(i)+'.mp4'
  !python predict.py --video_path={video} --load_weights={weights_path}

2022-06-02 13:55:03.763298: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Number of frames in video and dictionary are not the same!
Fail to load, predict only.
Beginning predicting......
Done......


### Test accuracy

In [11]:
file_name = 'Final_PremierQatar_Tie'
file_name = 'cleanVal/PremierRoma_Final_2_10'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights2/TrackNet-SoloPadelCleanAll-480p-1F'
point_n = 0
for point_n in range(1,4):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --frame_stack=1 --back_frame_stack=0 --HEIGHT=480 --WIDTH=720 --load_weights={weights_path} # 2>&1 | tee {res}

raw_data/points/cleanVal/PremierRoma_Final_2_10_1.mp4
2022-06-11 09:37:01.361621: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 46
Number of true negative: 25
Number of false positive FP1: 21
Number of false positive FP2: 7
Number of false negative: 36
Accuracy: 0.5259259259259259
Precision: 0.6216216216216216
Recall: 0.5609756097560976
Total Time: 19.779853582382202
(ACC + Pre + Rec)/3: 0.5695077191012151
Done......
raw_data/points/cleanVal/PremierRoma_Final_2_10_2.mp4
2022-06-11 09:38:41.858501: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Begin

### Train Padel

In [ ]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

In [16]:
!rm -rf train_data/*
!python video2img.py --HEIGHT=480 --WIDTH=720 --sigma=1

['raw_data/PremierRoma_Octavos_GRMS_2_53_0.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_1.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_2.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_5.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_0.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_4.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_4.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_5.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_0.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_4.mp4', 'raw_data/PremierRoma_Sem

#### Train sequencial (error en el orden de las imagenes)
*   TrackNet-PadelClean2: Entrenado solo con padel, pocos frames (5000) limpios (solo fragmentos con la bola en juego). --lr=10
*   TrackNet-PadelClean3: Continuación, 50 epochs --batch_size=5, con 7k frames (alcanza 0.956 en la validación. - loss:1.20e-4)
*   TrackNet-PadelClean4: Continuación, +50epochs --bacth_size=5, con 12k frames, --lr=1 (alcanza 0.94 en la validación. - loss:1.1*   TrackNet-PadelClean5: Continuación, +50epochs (800 steps), con 20k frames, --lr=1 (alcanza 0.93 en la validación. - loss:1.09e-4)8e-4)
-----
*   TrackNet-PadelCleanAll: Solo padel CleanDS, 25epochs (1k steps), con 20k frames, --lr=10 (alcanza 0.82 en la validación. - loss:1.80e-4)
*   TrackNet-PadelCleanAll1: Continuación, 50epochs (1k steps) --lr=10/1 (alcanza 0.86 en la validación. - loss:1.23e-4)
*   TrackNet-PadelCleanAll2: Continuación, 25epochs (1k steps) --lr=1 (alcanza 0.86 en la validación. - loss:1.08e-4)
*   TrackNet-PadelCleanAll3: Continuación, 50epochs (1k steps) --lr=1 (alcanza 0.88 en la validación. - loss:1.06e-4)




#### Train desordenado
*    TrackNet-SoloPadelCleanAll: solo padel CleanDS, 50epochs (800) (valTrain=0.9, loss=2.3e-4, valTest=0.8)
-------
*    TrackNet-SoloPadelCleanAll-720p: solo padel CleanDS (480p), 35epochs (800) (valTrain=0.9, loss=1.9e-4)
*    TrackNet-SoloPadelCleanAll-720p2: Continuación, +50epochs (1500) (valTrain=0.9, loss=1.75e-4)
-------
*    TrackNet-SoloPadelCleanAll-480p: solo padel CleanDS (480p) sigma=3, 30epoch (1500) (valTrain=0.9, loss=1.29e-4, cleanVal=0.9-0.94)
-------
*    TrackNet-SoloPadelCleanAll-480p-5F+2F: solo padel CleanDS (480p) sigma=3, 22epoch(1500) (valTrain=0.92, loss=1.25e-4, cleanVal=0.89-0.96 )
------
*    TrackNet-SoloPadelCleanAll-480p-9F+4F: solo padel CleanDS (480p) sigma=3, 50epoch(1500) (valTrain=0.94, loss=1.10e-4, cleanVal=0.91-0.96)
------
*    TrackNet-SoloPadelCleanAll-480p-1F: solo padel CleanDS (480p) sigma=2, 50epoch(1500) (valTrain=0.90, loss=0.947e-4, cleanVal=0.56-0.65)
------
*    TrackNet-SoloPadelCleanAll-480p-1s-1F: solo padel CleanDS (480p) sigma=1, 50epoch(1500) (valTrain=0.84, loss=0.727e-4, cleanVal=malo)


In [17]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights2/TrackNet-SoloPadelCleanAll-480p-1s-1F'
!python train2.py --save_weights={weights_path}  --epochs=50 --batch_size=2 --frame_stack=1 --back_frame_stack=0 --HEIGHT=480 --WIDTH=720 --lr=10 --tol=5 # --pre_trained=True  --load_weights={weights_path[:-1]}

lerning rate: 10.0
2022-06-10 22:22:52.892100: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 585s 380ms/step - loss: 2.8828e-04 - binary_accuracy: 0.9999
==========Epoch 0 start validation========== 400
Epoch 0 accuracy: 0.465
Epoch 0 precision: 0.606
Epoch 0 recall: 0.572
Epoch 0 average = (accuracy + precision + recall)/3: 0.548
==========Epoch 1, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 570s 380ms/step - loss: 1.7637e-04 - binary_accuracy: 0.9999
==========Epoch 2, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 571s 380ms/step - loss: 1.5280e-04 - binary_accuracy: 0.9999
==========Epoch 3, Train ste

### Debug

In [8]:
%load_ext autoreload
%autoreload 2

In [81]:
from utils import *
HEIGHT = 288
WIDTH = 512
BATCH_SIZE = 2
FRAME_STACK = 3
BACK_FRAME_STACK = 1

match_path = 'train_data'
match_list = [os.sep.join([os.getcwd(), match_path, match]) for match in os.listdir(match_path)]

x_train, x_test, y_train, y_test = split_train_test2(match_list, FRAME_STACK, BACK_FRAME_STACK, ratio=0.9, shuffle=True)
gen = data_generator2(BATCH_SIZE, x_train+x_test, y_train+y_test, FRAME_STACK, BACK_FRAME_STACK)

18572
2064
18572


In [64]:
x, y = next(gen)
print(x.shape)

(2, 3, 288, 512)


In [65]:
list(zip(x_train, y_train))[:3]

[('/content/TrackNet-Padel/train_data/match22/x_data/match22_PremierRoma_Semis_LGBC_1_00_6_105.jpg',
  '/content/TrackNet-Padel/train_data/match22/y_data/match22_PremierRoma_Semis_LGBC_1_00_6_105.jpg'),
 ('/content/TrackNet-Padel/train_data/match20/x_data/match20_PremierRoma_Semis_LGBC_1_00_2_606.jpg',
  '/content/TrackNet-Padel/train_data/match20/y_data/match20_PremierRoma_Semis_LGBC_1_00_2_606.jpg'),
 ('/content/TrackNet-Padel/train_data/match53/x_data/match53_PremierRoma_Semis_LGBC_2_35_2_117.jpg',
  '/content/TrackNet-Padel/train_data/match53/y_data/match53_PremierRoma_Semis_LGBC_2_35_2_117.jpg')]

In [89]:
im = cv2.imread('train_data/match1/x_data/match1_PremierRoma_Octavos_GRMS_2_53_0_0.jpg')
im.shape

(288, 512, 3)